# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
#!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

#### Call the ChatModel (the LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is a calming and soothing color that reminds me of the ocean and the sky. It brings a sense of peace and tranquility to me whenever I see it. I love how versatile blue is, from light shades like baby blue to deep navy blue. It's a color that never fails to make me feel relaxed and at ease.", response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 13, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-62031da4-db7a-483c-98f3-74b36f3cac8a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 68, 'total_tokens': 81})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have enough information to determine your favorite color. Can you please provide me with more details?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 13, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ff79da76-6694-4410-8f14-30149ebf1e2a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain_community

In [11]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [12]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [13]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [14]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [15]:
chain.invoke("HII!")

{'content': 'HII!',
 'messages': [HumanMessage(content='HII!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is prince'),
  AIMessage(content='Nice to meet you, Prince! How can I help you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Prince.')],
 'text': 'Hello again! How can I assist you today, Prince?'}

In [16]:
chain.invoke("my name is prince")    

{'content': 'my name is prince',
 'messages': [HumanMessage(content='HII!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is prince'),
  AIMessage(content='Nice to meet you, Prince! How can I help you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Prince.'),
  HumanMessage(content='HII!'),
  AIMessage(content='Hello again! How can I assist you today, Prince?')],
 'text': 'Yes, I remember your name is Prince. How can I assist you today?'}

In [17]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='HII!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is prince'),
  AIMessage(content='Nice to meet you, Prince! How can I help you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Prince.'),
  HumanMessage(content='HII!'),
  AIMessage(content='Hello again! How can I assist you today, Prince?'),
  HumanMessage(content='my name is prince'),
  AIMessage(content='Yes, I remember your name is Prince. How can I assist you today?')],
 'text': 'Your name is Prince. How can I assist you today?'}

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using chatGPT API.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-chatbot.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-chatbot.py

In [18]:
print("The End")

The End
